In [ ]:
import pandas as pd 
import numpy as np 
import datetime
import gc
import os
import random
import time
import warnings


import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost
import catboost
import seaborn as sns
from pandas import DataFrame
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve, auc
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
import matplotlib.pyplot as plt
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
from statsmodels.gam.tests.test_gam import sigmoid
from tqdm import tqdm



In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
train.shape,test.shape

In [ ]:
##vars
import random
label = 'target'
seed = 0
local_test = True
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(seed)
params = {
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        
        'n_jobs': -1,
        'learning_rate': 0.006,
        'num_leaves': 2 ** 8,
        'max_depth': 8,
        'tree_learner': 'serial',
        'colsample_bytree': 0.8,
        'subsample_freq': 1,
        'subsample': 0.8,

        'max_bin': 255,
        'verbose': -1,
        'seed': seed,

    }
base_features = [x for x in train.columns if 'cont' in x]
remove_features = [label,'id']

In [ ]:
train.head()

In [ ]:
## for reg 
sns.distplot(train['target'],bins=100,)
plt.show()

In [ ]:
train.target.sort_values(ascending=True).head()
# train = train[train['target']!=0].reset_index(drop=True)
train.shape

In [ ]:
# from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
import gc
# from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import KFold

def make_test(new_features):
    features = base_features + new_features
    oof_predictions = np.zeros(len(train))
    final_predictions = np.zeros(len(test))
    
    cv = KFold(n_splits=10,shuffle=True,random_state=seed)
    if local_test:
        n_estimators=1000
    else:
        n_estimators = 10000
    
    lgb = LGBMRegressor(**params,n_estimators=n_estimators,device='GPU')

    for n,(trn_id,val_id) in enumerate(cv.split(train[features],train[label])):
        print(f"===== training fold {n+1} =====")
        trn_x,trn_y = train.loc[trn_id,features],train.loc[trn_id,label]
        val_x,val_y = train.loc[val_id,features],train.loc[val_id,label]
        
        lgb.fit(trn_x,trn_y,eval_set=[(val_x,val_y)],early_stopping_rounds=100,verbose=-1)
        
        oof_predictions[val_id] = lgb.predict(val_x)
        final_predictions += lgb.predict(test[features]) / cv.n_splits
        

        mse_score = np.sqrt(mean_squared_error(y_true=val_y,y_pred=oof_predictions[val_id]))
                                       
        
        del trn_x,trn_y,val_x,val_y
        gc.collect()
    
    
    cur_mse_score = np.sqrt(mean_squared_error(y_true=train[label],y_pred=oof_predictions))
    cur_mae_score = mean_absolute_error(y_true=train[label],y_pred=oof_predictions)                                   
    print(f"global mse score {cur_mse_score}")
    print(f"global mae score {cur_mae_score}")
    print(f"diff with previous version {score[0] - cur_mse_score}")
    print(f"diff with previous version {score[1] - cur_mae_score}")
    if not local_test:
        test[label] = final_predictions
        test[['id',label]].to_csv(f'sub_{np.round(cur_mse_score,4)}.csv',index=False)
    
    return [cur_mse_score,cur_mae_score]

In [ ]:
#baseline
local_test=False
score = [0.6970820000536615, 0.5829603998473519]
make_test([])

## tuning WITH OPTUNA
